# Get Possible New Music, Events, Promos from Music Artists

## Tweepy API Get Tweets

In [1]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
 
import twitter_credentials
import numpy as np
import pandas as pd
import datetime as dt
import re


# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None,list_id=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_list=list_id
        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_list_tweets(self):
        tweets = []
        for tweet in Cursor(self.twitter_client.list_timeline, list_id=self.twitter_list).pages():
            tweets=tweet+tweets
        return tweets
    
    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['handle'] = np.array([tweet.user.screen_name for tweet in tweets])
        df['name'] = np.array([tweet.user.name for tweet in tweets])

        return df

In [2]:
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_rows', 100)

## My Lists

In [3]:
popular_artist_list='1255643235386605569'
rkm_artist_list='1090078355246985217'
chill_artist_list='1256130288322842624'

## Assign Lists

In [4]:
twitter_popular_list = TwitterClient(None,popular_artist_list)
twitter_rkm_list = TwitterClient(None,rkm_artist_list)
twitter_chill_list=TwitterClient(None,chill_artist_list)
tweet_analyzer=TweetAnalyzer()

tweets_pop = twitter_popular_list.get_list_tweets()
tweets_rkm=twitter_rkm_list.get_list_tweets()
tweets_chilled_cheese=twitter_chill_list.get_list_tweets()

popular_artists=tweet_analyzer.tweets_to_data_frame(tweets_pop)
chill_artists=tweet_analyzer.tweets_to_data_frame(tweets_chilled_cheese)
rkm_artists=tweet_analyzer.tweets_to_data_frame(tweets_rkm)

rkm_tweets=rkm_artists.Tweets
popular_tweets=popular_artists.Tweets
chilled_tweets=chill_artists.Tweets

## Filter Tweets for New music, promos, and event updates

In [12]:
show_tweets=[]
checks=['.{}'.format(dt.date.today().strftime("%m")[1:]),
        '/{}'.format(dt.date.today().strftime("%m")[1:]),
        '-{}'.format(dt.date.today().strftime("%m")[1:]),
        '/{}'.format((dt.date.today()+pd.DateOffset(weeks=1)).strftime("%m")[1:]),
        '.{}'.format((dt.date.today()+pd.DateOffset(weeks=1)).strftime("%m")[1:]),
        '-{}'.format((dt.date.today()+pd.DateOffset(weeks=1)).strftime("%m")[1:])
       ]

month=dt.date.today().strftime("%B")[:3].lower()
month_week_ahead=(dt.date.today()+pd.DateOffset(weeks=1)).strftime("%B")[:3].lower()
year='{}'.format(dt.date.today().year)

for tweet in popular_tweets:
    flag=False
    if tweet.find('https://t.co')!=-1:
        tweet=tweet[:tweet.find('https://t.co')].lower()
    else:
        tweet=tweet.lower()    
    for check in checks:
        if check in tweet or bool(re.match("([\s\S]*[0-9]|0[0-9]|1[0-9]|2[0-3])\s*([AaPp][Mm])", tweet)) \
        or bool(re.match(" ([\s\S]*[0-9]|0[0-9]|1[0-9]|2[0-3]):([0-5][0-9])\s*([AaPp][Mm])", tweet)) \
        or bool(re.match("[\s\S]*(([\s\W]mon|[\s\W]tues|[\s\W]wed(nes)?|[\s\W]thur(s)?|[\s\W]fri|[\s\W]sat(ur)?|[\s\W]sun)(day)?[\s\W])",tweet))\
        or bool(re.search(rf"\b({month}|{month_week_ahead}|{year}|free|days|ep|single|new music|is out|out now|out in|remix|new song|tomorrow|release|released|single|coming soon|listen to|this month|drops|next week|download|presave|pre-save|available|music video|listen on|album|noon|midnight|download|soundcloud|spotify|youtube)\b",tweet)):
            show_tweets.append(True)
            flag=True
            break
    if flag==False:
        show_tweets.append(False)

In [13]:
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_rows', 200)

final=popular_artists[show_tweets][['Tweets','name','handle','date']].sort_values('date',ascending=False).reset_index(drop=True)

In [14]:
final_no_retweets=final[final.Tweets.apply(lambda x: False if x[:2]=='RT' else True)].sort_values('date',ascending=False).reset_index(drop=True)
final_retweets=final[final.Tweets.apply(lambda x: True if x[:2]=='RT' else False)].sort_values('date',ascending=False).reset_index(drop=True)
final_retweets=final_retweets.copy()
final_retweets['Tweets']=final_retweets.Tweets.apply(lambda x:x[x.find(':')+1:])

## Possible Event, Promo, or New music announcement w/o Retweets

In [15]:
final_no_retweets

,Tweets,name,handle,date
0,"Check out the latest remix of ""In Your Eyes"" by @NickyRomero! 🔥👁https://t.co/RLqu0eAjOm https://t.co/o5j6Zn7oWC",Robin Schulz,robin_schulz,2020-05-08 15:15:00
1,Part 2 of the listening party tonight at 6pm CET🖤✨\n\nhttps://t.co/9AhXtU1oVa https://t.co/j7vHrmu6gq,dagnymusic,DagnyMusic,2020-05-08 15:04:58
2,Mother’s Day Sunday 😳,Lil Baby,lilbaby4PF,2020-05-08 14:01:38
3,Feeling so nostalgic for shows lately so here’s a few tracks from opening night of GIRL: The World Tour. Listen to… https://t.co/or0OIQPV3i,MAREN MORRIS,MarenMorris,2020-05-08 14:01:11
4,The animated video for Break My Heart is now live on my YouTube! 💜🌕🚀 https://t.co/h7gTM21r8o https://t.co/k3xHENIE2e,DUA LIPA,DUALIPA,2020-05-08 13:59:19
5,Free Big Meech,Lil Baby,lilbaby4PF,2020-05-08 13:56:50
6,Petals For Armor official release day promo vibes: https://t.co/2s75QAkMvA,hayley from Paramore 🌺,yelyahwilliams,2020-05-08 13:32:37
7,.@iheartradio is throwing a prom and I’m the DJ! Tune it tonight @ 8pm ET it’s going to be lit 🔥… https://t.co/innfKrLjM4,marshmello,marshmellomusic,2020-05-08 13:03:26
8,new @beatsbynav album executive produced by me and my brother @cashup featuring some of our close friends. Proud of… https://t.co/3QwZIepcN4,The Weeknd,theweeknd,2020-05-08 04:43:03
9,okay friends. it’s here for everyone. \n\ngonna go celebrate this lil victory and i’ll be around more tomorrow 🌺🙏🏼 gn… https://t.co/EFHpQXFlJV,hayley from Paramore 🌺,yelyahwilliams,2020-05-08 04:25:12


## Possible Event, Promo, or New music announcement Only Retweets

In [16]:
final_retweets

,Tweets,name,handle,date
0,.@yelyahwilliams' #PetalsForArmor is finally here!! What's your favorite song off the album? 🌸 \n\n🎨: @seancellmore #MTVxSean https:…,hayley from Paramore 🌺,yelyahwilliams,2020-05-08 15:15:28
1,"This week on ""Inside With The Advocate,"" R&amp;B star @4everBrandy talks about her career, her new single #BabyMama, and he…",b r a n d y,4everBrandy,2020-05-08 15:08:34
2,One day it just cliqued. PUMA x @TheHundreds out now: https://t.co/z0bVBEg2Tn https://t.co/ySlYCVAysZ,Meek Mill,MeekMill,2020-05-08 14:34:19
3,"“As soon as I heard the song, I was instantly obsessed.”\n@marshmellomusic connects with @halsey for #BeKind.\nListen on #ALi…",marshmello,marshmellomusic,2020-05-08 13:01:22
4,“I FEEL IT” OUT NOW! https://t.co/PDrUdaC6wG https://t.co/BcgPv8uMm2,OVO Sound,OVOSound,2020-05-08 11:20:39
5,Here's Thursday's #RDTop3! \n1. @Alesso - #Midnight (f. @LiamPayne)\n2. @KellyClarkson - #IDareYou\n3. @R3HAB - #GoodExample…,Liam,LiamPayne,2020-05-08 08:17:26
6,HEARTBREAK ANTHEM WITH @gnash OUT NOW https://t.co/j5iqgkL1wF https://t.co/GqLyIEENlO,Garrett Nash 🌻,gnash,2020-05-08 05:27:11
7,🔥OUT NOW🔥Check out @bujubanton &amp; @johnlegend's new single #MEMORIES. Listen now on @TIDAL! https://t.co/waO0IFzsj4 https://t…,DJ KHALED,djkhaled,2020-05-08 04:04:52
8,Listening to @iamjojo new album #GoodtoKnow as a whole in sequence is something else. I’m shook. ☘️🧡,JoJo.,iamjojo,2020-05-08 03:04:44
9,jojo’s new album makes me want to invite over my local fb but also text my ex to see what he’s up to. @iamjojo,JoJo.,iamjojo,2020-05-08 03:04:14


In [17]:
#'new music','is out','out soon','out now','new song','tomorrow','new release',' release'
        #'new single','coming soon','listen to','this month',' single'
        #,'more days','drops','next week','free','download','presave','pre-save','available','days'
        #,'music video','listen on','album',' noon','midnight','download','soundcloud','spotify',
        #,dt.date.today().strftime("%B")[:3].lower()
        #,(dt.date.today()+pd.DateOffset(weeks=1)).strftime("%B")[:3].lower(),